<a href="https://colab.research.google.com/github/Arshad221b/ASL-new/blob/main/sarcasm_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [144]:
! pip install kaggle
from google.colab import files 
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"arshadsk","key":"d503971701e4db050f9b913da7842a12"}'}

In [145]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [146]:
!kaggle datasets download -d rmisra/news-headlines-dataset-for-sarcasm-detection

news-headlines-dataset-for-sarcasm-detection.zip: Skipping, found more recently modified local copy (use --force to force download)


In [165]:
! unzip news-headlines-dataset-for-sarcasm-detection.zip -d sarcasm

Archive:  news-headlines-dataset-for-sarcasm-detection.zip
replace sarcasm/Sarcasm_Headlines_Dataset.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [166]:
import pandas as pd
import numpy as np
import tensorflow
import matplotlib.pyplot as plt

In [167]:
data = pd.read_json("/content/sarcasm/Sarcasm_Headlines_Dataset.json", lines = True)

In [168]:
data.head(10)

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0
5,https://www.huffingtonpost.com/entry/advancing...,advancing the world's women,0
6,https://www.huffingtonpost.com/entry/how-meat-...,the fascinating case for eating lab-grown meat,0
7,https://www.huffingtonpost.com/entry/boxed-col...,"this ceo will send your kids to school, if you...",0
8,https://politics.theonion.com/top-snake-handle...,top snake handler leaves sinking huckabee camp...,1
9,https://www.huffingtonpost.com/entry/fridays-m...,friday's morning email: inside trump's presser...,0


In [169]:
data2 = pd.read_json("/content/sarcasm/Sarcasm_Headlines_Dataset_v2.json", lines = True)

In [170]:
data2 = data2[["article_link","headline", "is_sarcastic"]]

In [171]:
data2.head(10)

,article_link,headline,is_sarcastic
0,https://www.theonion.com/thirtysomething-scien...,thirtysomething scientists unveil doomsday clo...,1
1,https://www.huffingtonpost.com/entry/donna-edw...,dem rep. totally nails why congress is falling...,0
2,https://www.huffingtonpost.com/entry/eat-your-...,eat your veggies: 9 deliciously different recipes,0
3,https://local.theonion.com/inclement-weather-p...,inclement weather prevents liar from getting t...,1
4,https://www.theonion.com/mother-comes-pretty-c...,mother comes pretty close to using word 'strea...,1
5,https://www.huffingtonpost.com/entry/my-white-...,my white inheritance,0
6,https://www.huffingtonpost.com/entry/5-ways-to...,5 ways to file your taxes with less stress,0
7,https://www.theonion.com/richard-bransons-glob...,richard branson's global-warming donation near...,1
8,https://politics.theonion.com/shadow-governmen...,shadow government getting too large to meet in...,1
9,https://www.huffingtonpost.comhttp://pubx.co/6...,lots of parents know this scenario,0


In [172]:
data = pd.concat([data, data2])

In [173]:
data.head(10)

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0
5,https://www.huffingtonpost.com/entry/advancing...,advancing the world's women,0
6,https://www.huffingtonpost.com/entry/how-meat-...,the fascinating case for eating lab-grown meat,0
7,https://www.huffingtonpost.com/entry/boxed-col...,"this ceo will send your kids to school, if you...",0
8,https://politics.theonion.com/top-snake-handle...,top snake handler leaves sinking huckabee camp...,1
9,https://www.huffingtonpost.com/entry/fridays-m...,friday's morning email: inside trump's presser...,0


In [174]:
data.drop(['article_link'], axis= 1, inplace = True)
data.head(10)


,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0
5,advancing the world's women,0
6,the fascinating case for eating lab-grown meat,0
7,"this ceo will send your kids to school, if you...",0
8,top snake handler leaves sinking huckabee camp...,1
9,friday's morning email: inside trump's presser...,0


In [175]:
data.reset_index(drop = True, inplace =True)

In [176]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Dense, Bidirectional, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam 
import numpy as np

In [177]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['headline'])

In [215]:
total_words = len(tokenizer.word_index)

In [179]:

def applyToken(s):
  tokens = tokenizer.texts_to_sequences(s)[0]
  return tokens
  # print(tokens)

data['token'] = [applyToken([x]) for x in data['headline']]

# applyToken(data['headline'])
# print(data['headline'][0])
# print(max_len)

In [180]:
max_len = max([len(x) for x in data['token']])
print(max_len)

152


In [204]:
padded = np.array(pad_sequences(data['token'], maxlen = max_len, padding = 'pre') )

In [205]:
padded

array([[    0,     0,     0, ...,     5,  2722,  8922],
       [    0,     0,     0, ...,   253,     8,  1059],
       [    0,     0,     0, ...,    44,     1, 11334],
       ...,
       [    0,     0,     0, ...,     6,   791,  1833],
       [    0,     0,     0, ...,  2618,   840,  6574],
       [    0,     0,     0, ...,     6,   255,   186]], dtype=int32)

In [221]:
len(padded)

55328

In [228]:
split_train = int(0.8* len(padded))
train_x = padded[:split_train]
val_x = padded[split_train:]
train_y =  data['is_sarcastic'][:split_train]
val_y = data['is_sarcastic'][split_train:]

In [227]:
xs = np.asarray(data['token_pad'])
ys = data['is_sarcastic']

In [234]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words +1, 16,),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()
#earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
history = model.fit(train_x, train_y
                              ,epochs=5, validation_data=(val_x, val_y),verbose=1)

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_21 (Embedding)     (None, None, 16)          494160    
_________________________________________________________________
bidirectional_21 (Bidirectio (None, 64)                9600      
_________________________________________________________________
dense_34 (Dense)             (None, 6)                 390       
_________________________________________________________________
dense_35 (Dense)             (None, 1)                 7         
Total params: 504,157
Trainable params: 504,157
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
1384/1384 [==============================] - 26s 17ms/step - loss: 0.4891 - accuracy: 0.7253 - val_loss: 0.1889 - val_accuracy: 0.9296
Epoch 2/5
1384/1384 [==============================] - 24s 17ms/step - loss: 0.1276 - accuracy: 0.95

In [236]:
model.save('my_checkpoint.h5')

In [233]:
model.load_weights('./checkpoints/my_checkpoint')

In [283]:
s1 = 'Cows lose their jobs as milk prices drop'
tokenizer.fit_on_texts(s1)
s1 = tokenizer.texts_to_sequences([s1])
s1 = pad_sequences(s1, maxlen = max_len, padding = 'pre')
if model.predict(s1)>= 0.75:
  print("Sarcastic headline!")
else:
  print('Not sarcastic')

Sarcastic headline!
